In [ ]:
import re
import sys
from itertools import chain
import numpy as np

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

In [ ]:
def get_objects(app, pattern, keep_out_of_service=False):
    objs = app.GetCalcRelevantObjects(pattern)
    if keep_out_of_service:
        return objs
    return [obj for obj in objs if not obj.outserv]

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\ENTSOE_20221018'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
generators = get_objects(app, '*.ElmSym')
loads = get_objects(app, '*.ElmLod')
loads = [load for load in loads if load.plini > 0 and load.qlini > 0]
transformers = get_objects(app, '*.ElmTr2')
shunts = get_objects(app, '*.ElmShnt')
capacitors = get_objects(app, '*.ElmScap')
impedances = get_objects(app, '*.ElmZpu')
lines = get_objects(app, '*.ElmLne')
n_generators, n_loads, n_transformers = len(generators), len(loads), len(transformers)
n_shunts, n_capacitors, n_impedances = len(shunts), len(capacitors), len(impedances)
n_lines = len(lines)
print(f'There are {n_generators} generators.')
print(f'There are {n_loads} loads.')
print(f'There are {n_shunts} shunts.')
print(f'There are {n_capacitors} series capacitors.')
print(f'There are {n_impedances} common impedances.')
print(f'There are {n_lines} lines.')
print(f'There are {n_transformers} transformers.')

In [ ]:
bus_names = []
for obj in chain(lines, capacitors, impedances):
    for i in 1,2:
        bus_name = obj.GetAttribute(f'bus{i}').cterm.loc_name
        if bus_name not in bus_names:
            bus_names.append(bus_name)
for obj in chain(generators, loads, shunts):
    bus_name = obj.bus1.cterm.loc_name
    if bus_name not in bus_names:
        bus_names.append(bus_name)
for obj in transformers:
    for c in 'hl':
        bus_name = obj.GetAttribute(f'bus{c}v').cterm.loc_name
        if bus_name not in bus_names:
            bus_names.append(bus_name)
all_in_service_buses = get_objects(app, '*.ElmTerm')
buses = [bus for bus in all_in_service_buses if bus.loc_name in bus_names]
n_buses = len(buses)
print(f'There are {n_buses} buses.')

In [ ]:
load_flow = app.GetFromStudyCase('ComLdf')
err = load_flow.Execute()

get_name = lambda elem: elem.name

powergenerators = sorted([PowerGenerator(gen) for gen in generators], key=get_name)
powerloads = sorted([PowerLoad(load) for load in loads], key=get_name)
powerbuses = sorted([PowerBus(bus) for bus in buses], key=get_name)
powertransformers = sorted([PowerTransformer(transformer, voltages_from='bus') for transformer in transformers], key=get_name)
powerlines = sorted([PowerLine(line) for line in lines], key=get_name)
powershunts = sorted([Shunt(shunt) for shunt in shunts], key=get_name)
powercapacitors = sorted([SeriesCapacitor(cap) for cap in capacitors], key=get_name)
powerimpedances = sorted([CommonImpedance(imp) for imp in impedances], key=get_name)

In [ ]:
with open('ENTSOE.inc', 'w') as fid:
    for gen in powergenerators:
        fid.write(str(gen) + '\n')
    fid.write('\n')
    for load in powerloads:
        fid.write(str(load) + '\n')
    fid.write('\n')
    for line in powerlines:
        fid.write(str(line) + '\n')
    fid.write('\n')
    for trans in powertransformers:
        fid.write(str(trans) + '\n')
    fid.write('\n')
    for bus in powerbuses:
        fid.write(str(bus) + '\n')
    fid.write('\n')
    for shunt in powershunts:
        fid.write(str(shunt) + '\n')
    fid.write('\n')
    for cap in powercapacitors:
        fid.write(str(cap) + '\n')
    fid.write('\n')
    for imp in powerimpedances:
        fid.write(str(imp) + '\n')